In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

from datetime import datetime

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as Layer

import tensorboard
import matplotlib.pyplot as plt

import numpy as np

In [3]:
# load the dataset and normalize
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
train_images = train_images / 255.0

x_train = train_images.reshape(-1, 28, 28, 1) #add an additional dimension to represent the single-channel
x_test = test_images.reshape(-1, 28, 28, 1)

np.random.seed(1)
tf.random.set_seed(1)

In [36]:
regular_model = tf.keras.models.Sequential(name='CNN_Increasing_Filters')

regular_model.add(Layer.Conv2D(8, (3, 3), padding='same', activation='relu', name='Conv_Layer_1')) 
regular_model.add(Layer.Conv2D(9, (3, 3), padding='same', activation='relu', name='Conv_Layer_2'))
regular_model.add(Layer.Conv2D(10, (3, 3),padding='same', activation='relu', name='Conv_Layer_3'))
regular_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2), name='Pooling_Layer_14x14'))

regular_model.add(Layer.Conv2D(20, (3, 3), padding='same', activation='relu', name='Conv_Layer_4'))
regular_model.add(Layer.Conv2D(21, (3, 3), padding='same', activation='relu', name='Conv_Layer_5'))
regular_model.add(Layer.Conv2D(22, (3, 3),padding='same', activation='relu', name='Conv_Layer_6'))
regular_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2), name='Pooling_Layer_7x7'))

regular_model.add(Layer.Conv2D(44, (3, 3), padding='same', activation='relu', name='Conv_Layer_7'))
regular_model.add(Layer.Conv2D(45, (3, 3), padding='same', activation='relu', name='Conv_Layer_8'))
regular_model.add(Layer.Conv2D(46, (3, 3), padding='same', activation='relu', name='Conv_Layer_9'))
regular_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2), name='Pooling_Layer_3x3'))

regular_model.add(Layer.Conv2D(92, (3, 3), padding='same', activation='relu', name='Conv_Layer_10'))

regular_model.add(Layer.Flatten())
regular_model.add(Layer.Dense(184))
regular_model.add(Layer.Activation('relu'))
regular_model.add(Layer.Dense(10))
regular_model.add(Layer.Activation('softmax'))

regular_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=.001), metrics=['accuracy'])
# regular_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=.001), metrics=['accuracy'])
# regular_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=.001), metrics=['accuracy'])
regular_model.build(input_shape=(1,28,28,1))
# regular_model.summary()


Model: "CNN_Increasing_Filters"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_Layer_1 (Conv2D)        (1, 28, 28, 8)            80        
_________________________________________________________________
Conv_Layer_2 (Conv2D)        (1, 28, 28, 9)            657       
_________________________________________________________________
Conv_Layer_3 (Conv2D)        (1, 28, 28, 10)           820       
_________________________________________________________________
Pooling_Layer_14x14 (MaxPool (1, 14, 14, 10)           0         
_________________________________________________________________
Conv_Layer_4 (Conv2D)        (1, 14, 14, 20)           1820      
_________________________________________________________________
Conv_Layer_5 (Conv2D)        (1, 14, 14, 21)           3801      
_________________________________________________________________
Conv_Layer_6 (Conv2D)        (1, 14, 14, 22)

In [37]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
regular_model.fit(
    x_train,
    train_labels, 
    batch_size=512,
    epochs=1,
    callbacks=[tensorboard_callback])

# Evaluate
score = regular_model.evaluate(x_test, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# ------------------------------ learning rates ------------------------------ #
# .01
# Test loss: 2.326310873031616
# Test accuracy: 0.10809999704360962

# .001
# Test loss: 18.132125854492188
# Test accuracy: 0.9538000226020813

# .0001
# Test loss: 70.65046691894531
# Test accuracy: 0.7976999878883362


# -------------------------------- batch sizes ------------------------------- #
# 64
# Test loss: 4.986489295959473
# Test accuracy: 0.9850000143051147

# 128
# Test loss: 13.246450424194336
# Test accuracy: 0.9714999794960022

# 256
# Test loss: 9.630948066711426
# Test accuracy: 0.9751999974250793

# 512
# Test loss: 16.442401885986328
# Test accuracy: 0.9625999927520752

# -------------------------------- optimizers -------------------------------- #
# Adam
# Test loss: 14.913040161132812
# Test accuracy: 0.9635000228881836

# SGD
# Test loss: 2.6059443950653076
# Test accuracy: 0.0885000005364418

# RMSprop
# Test loss: 17.7608585357666
# Test accuracy: 0.9556999802589417


313/313 [==============================] - 3s 9ms/step - loss: 17.7609 - accuracy: 0.9557
Test loss: 17.7608585357666
Test accuracy: 0.9556999802589417


In [30]:
inverted_model = tf.keras.models.Sequential(name='CNN_Decreasing_Filters')

inverted_model.add(Layer.Conv2D(100, (3, 3), padding='same', activation='relu')) 
inverted_model.add(Layer.Conv2D(99, (3, 3), padding='same', activation='relu'))
inverted_model.add(Layer.Conv2D(98, (3, 3),padding='same', activation='relu'))
inverted_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

inverted_model.add(Layer.Conv2D(49, (3, 3), padding='same', activation='relu'))
inverted_model.add(Layer.Conv2D(48, (3, 3), padding='same', activation='relu'))
inverted_model.add(Layer.Conv2D(47, (3, 3),padding='same', activation='relu'))
inverted_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

inverted_model.add(Layer.Conv2D(24, (3, 3), padding='same', activation='relu'))
inverted_model.add(Layer.Conv2D(23, (3, 3), padding='same', activation='relu'))
inverted_model.add(Layer.Conv2D(22, (3, 3), padding='same', activation='relu'))
inverted_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

inverted_model.add(Layer.Conv2D(11, (3, 3), padding='same', activation='relu'))

inverted_model.add(Layer.Flatten())
inverted_model.add(Layer.Dense(128))
inverted_model.add(Layer.Activation('relu'))
inverted_model.add(Layer.Dense(10))
inverted_model.add(Layer.Activation('softmax'))

inverted_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=.001), metrics=['accuracy'])
# inverted_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=.001), metrics=['accuracy'])
# inverted_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=.001), metrics=['accuracy'])

inverted_model.build(input_shape=(1,28,28,1))
# inverted_model.summary()

Model: "CNN_Decreasing_Filters"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_140 (Conv2D)          (1, 28, 28, 100)          1000      
_________________________________________________________________
conv2d_141 (Conv2D)          (1, 28, 28, 99)           89199     
_________________________________________________________________
conv2d_142 (Conv2D)          (1, 28, 28, 98)           87416     
_________________________________________________________________
max_pooling2d_42 (MaxPooling (1, 14, 14, 98)           0         
_________________________________________________________________
conv2d_143 (Conv2D)          (1, 14, 14, 49)           43267     
_________________________________________________________________
conv2d_144 (Conv2D)          (1, 14, 14, 48)           21216     
_________________________________________________________________
conv2d_145 (Conv2D)          (1, 14, 14, 47)

In [31]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
inverted_model.fit(
    x_train,
    train_labels, 
    batch_size=512,
    epochs=1,
    callbacks=[tensorboard_callback])

# Evaluate
score = inverted_model.evaluate(x_test, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# ------------------------------ learning rates ------------------------------ #
# .01
# Test loss: 13.599109649658203
# Test accuracy: 0.9607999920845032


# .001
# Test loss: 21.251075744628906
# Test accuracy: 0.9520999789237976


# -------------------------------- batch sizes ------------------------------- #
# 256
# Test loss: 22.917844772338867
# Test accuracy: 0.10090000182390213


# 512
# Test loss: 133.16815185546875
# Test accuracy: 0.6804999709129333

# -------------------------------- optimizers -------------------------------- #
# Adam
# Test loss: 14.106782913208008
# Test accuracy: 0.9668999910354614

# SGD
# Test loss: 2.349264144897461
# Test accuracy: 0.10029999911785126

# RMSprop
# Test loss: 75.77559661865234
# Test accuracy: 0.8079000115394592


313/313 [==============================] - 11s 35ms/step - loss: 14.1068 - accuracy: 0.9669
Test loss: 14.106782913208008
Test accuracy: 0.9668999910354614


In [21]:
hourglass_model = tf.keras.models.Sequential(name='CNN_HourGlass_Filters')

hourglass_model.add(Layer.Conv2D(30, (3, 3), padding='same', activation='relu')) 
hourglass_model.add(Layer.Conv2D(31, (3, 3), padding='same', activation='relu'))
hourglass_model.add(Layer.Conv2D(32, (3, 3),padding='same', activation='relu'))
hourglass_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

hourglass_model.add(Layer.Conv2D(64, (3, 3), padding='same', activation='relu'))
hourglass_model.add(Layer.Conv2D(65, (3, 3), padding='same', activation='relu'))
hourglass_model.add(Layer.Conv2D(66, (3, 3),padding='same', activation='relu'))
hourglass_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

hourglass_model.add(Layer.Conv2D(65, (3, 3), padding='same', activation='relu'))
hourglass_model.add(Layer.Conv2D(64, (3, 3), padding='same', activation='relu'))
hourglass_model.add(Layer.Conv2D(63, (3, 3), padding='same', activation='relu'))
hourglass_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

hourglass_model.add(Layer.Conv2D(32, (3, 3), padding='same', activation='relu'))

hourglass_model.add(Layer.Flatten())
hourglass_model.add(Layer.Dense(128))
hourglass_model.add(Layer.Activation('relu'))
hourglass_model.add(Layer.Dense(10))
hourglass_model.add(Layer.Activation('softmax'))

hourglass_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=.001), metrics=['accuracy'])
# hourglass_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=.001), metrics=['accuracy'])
# hourglass_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=.001), metrics=['accuracy'])


hourglass_model.build(input_shape=(1,28,28,1))
# hourglass_model.summary()

Model: "CNN_HourGlass_Filters"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_100 (Conv2D)          (1, 28, 28, 30)           300       
_________________________________________________________________
conv2d_101 (Conv2D)          (1, 28, 28, 31)           8401      
_________________________________________________________________
conv2d_102 (Conv2D)          (1, 28, 28, 32)           8960      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (1, 14, 14, 32)           0         
_________________________________________________________________
conv2d_103 (Conv2D)          (1, 14, 14, 64)           18496     
_________________________________________________________________
conv2d_104 (Conv2D)          (1, 14, 14, 65)           37505     
_________________________________________________________________
conv2d_105 (Conv2D)          (1, 14, 14, 66) 

In [23]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
hourglass_model.fit(
    x_train,
    train_labels, 
    batch_size=512,
    epochs=1,
    callbacks=[tensorboard_callback])

# Evaluate
score = hourglass_model.evaluate(x_test, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# ------------------------------ learning rates ------------------------------ #
# .01
# Test loss: 7.139647483825684
# Test accuracy: 0.09489999711513519


# .001
# Test loss: 12.76994800567627
# Test accuracy: 0.9690999984741211

# .0001
# Test loss: 45.62290954589844
# Test accuracy: 0.8917999863624573

# -------------------------------- batch sizes ------------------------------- #
# 128
# Test loss: 6.611076354980469
# Test accuracy: 0.9850999712944031

# 256
# Test loss: 6.701323509216309
# Test accuracy: 0.9804999828338623


# 512
# Test loss: 12.76994800567627
# Test accuracy: 0.9690999984741211

# -------------------------------- optimizers -------------------------------- #
# Adam
# Test loss: 12.76994800567627
# Test accuracy: 0.9690999984741211

# SGD
# Test loss: 2.392406463623047
# Test accuracy: 0.17710000276565552

# RMSprop
# Test loss: 24.464920043945312
# Test accuracy: 0.9332000017166138



313/313 [==============================] - 6s 18ms/step - loss: 3.6071 - accuracy: 0.1140
Test loss: 3.6071128845214844
Test accuracy: 0.11400000005960464
